In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.optimize import curve_fit

In [3]:
from itertools import product

In [4]:
from commons import smoothen, lse, avgnb

In [5]:
n_iterations = 20

# Preparations

## Loading the data

We start by loading the growth curves $N_i(t)$ while smoothening them ; additionally, we load it as a 38 x 54 grid where the inner 32 x 48 will contain the growth curves :

In [6]:
_ = np.moveaxis(np.load("synthetic-data.npy"), 1, 3)
Nt = np.zeros(np.array(_.shape) + np.array((0, 6, 6, 0)))
Nt[:, 3:-3, 3:-3, :] = _

Let's also calculate already the derivatives $\frac{\Delta N_i(t)}{\Delta t} = \Delta N_i(t) = N_i(t+1) - N_i(t)$

In [7]:
dNdt = Nt[:, 3:-3, 3:-3, 1:] - Nt[:, 3:-3, 3:-3, :-1]

and $\rho_i(t) = \frac{\Delta N_i(t)}{N_i(t)}$

In [8]:
rho = np.zeros_like(Nt[..., :-1])
rho[:, 3:-3, 3:-3] = dNdt / Nt[:, 3:-3, 3:-3, :-1]

Let's get the number of points consistent between $N_i(t)$ and the derivatives :

In [9]:
Nt = Nt[..., :-1]

## Dimensions

We also get the dimensions

In [10]:
n_plates, n_rows, n_columns, n_points = Nt.shape
plates, rows, columns, points = map(np.arange, Nt.shape)

## Initial $\alpha_i(t)$ parameter values

The diffusion models requires us to provide $\alpha_it(t)$ parameter values, which we load here :

In [11]:
alpha = np.zeros_like(rho)
alpha[:, 3:-3, 3:-3] = np.load("alpha/computed.npy")[..., :-11]

As we will recompute these values iteratively, along with the parameters that lead for those values, so here are some initial parameter values :

In [12]:
alphas = pd.read_csv("alpha/params.csv")
alphas.index = pd.MultiIndex.from_frame(alphas[["plate", "row", "column"]])
alphas = alphas[["r0 i", "c i", "m i"]]

The $r_0$ and $m$ parameters are global

In [13]:
r0_m = pd.DataFrame(index = pd.Index(plates, name = "plate"))

r0_m["r0"] = alphas["r0 i"].unique()
r0_m["m"] = alphas["m i"].unique()

while the $c_i$ parameters are population-specific

In [14]:
c_i = alphas["c i"]
c_i.index = alphas.index

# $\rho_i(t) = \alpha_i(t) \; f(s(t, x_i))$

$$
    \frac{dN}{dt} = \alpha(t) f(s) \; N(t)
$$
$$
    \frac{ds}{dt} = D \; \nabla^2 s(t) - \nu_1 \frac{dN}{dt} - \nu_2 N(t)
$$

where $f(s) = \left(1 + e^{-sK}\right)^{-\kappa}$.

By plugging the first equation into the second, and applying the mean-field approach, we obtain :
$$
    \frac{ds}{dt} = D \left(\bar s - s\right) - \nu_1 \alpha f(s) N - \nu_2 N
$$

We start by loading initial parameters for the model, as the fit is difficult enough :

In [15]:
params = pd.read_csv("diffusion-parameters/initial-guesses.csv")

We now perform the fit iteratively, while refitting the $\alpha_i(t)$ every iteration :

In [16]:
def f(s, K, kappa):
    return (1 + np.exp(-K*s)) ** (-kappa)

def dNN(a, s, K, kappa):
    return a * f(s, K, kappa)

def calculate_s(D, nu1, nu2, K, kappa, a, nt, m, s0 = 1):
    s = np.empty_like(a)
    s[..., 0] = s0
    
    for t in range(1, s.shape[-1]):
        diffusion = D * (m[..., t-1] - s[..., t-1])
        consumption1 = nu1 * a[..., t-1] * f(s[..., t-1], K, kappa) * nt[..., t-1]
        consumption2 = nu2 * nt[..., t-1]
        s[..., t] = s[..., t-1] + diffusion - consumption1 - consumption2
    
    return s

def calculate_m(s):
    m = np.empty_like(s)
    nr, nc = s.shape[:2]
    
    for r, c in product(rows, columns):
        val, n = 0, 0
        for i in range(max(0, r-1), min(nr, r+2)):
            for j in range(max(0, c-1), min(nc, c+2)):
                val += s[i, j]
                n += 1
        m[r, c] = (val - s[r, c]) / (n-1)
    
    return m

def iteration(D, nu1, nu2, K, kappa, a, nt, r, s0 = 1):
    m = np.full_like(a, s0)
    prev = np.inf
    
    for it in range(50):
        s = calculate_s(D, nu1, nu2, K, kappa, a, nt, m, s0)
        current = lse(r.reshape(-1), dNN(a, s, K, kappa).reshape(-1))
        if prev < current:
            break
        else:
            m = calculate_m(s)
            prev = current
    
    return s

In [17]:
previous_score = np.inf
for it in range(n_iterations):
    print(f"iteration {it+1}")
    previous_params = params.copy()
    for p in plates:
        print(f"\tplate {p+1}")
        params.loc[p] = curve_fit(
            f      = lambda _, d, nu1, k, kappa, nu2: dNN(alpha[p], iteration(nt = Nt[p], a = alpha[p], r = rho[p], D = d, nu1 = nu1, K = k, kappa = kappa, nu2 = nu2), K = k, kappa = kappa).reshape(-1),
            xdata  = 42,
            ydata  = rho[p].reshape(-1),
            bounds = (0, np.inf),
            p0     = params.loc[p]
        )[0]

    print("recalculating the alpha parameters")
        
    fs = np.array([
        f(
            s = iteration(nt = Nt[p], a = alpha[p], r = rho[p], **params.loc[p])[3:-3, 3:-3],
            K = params.loc[p, "K"],
            kappa = params.loc[p, "kappa"]
        )
        for p in plates
    ])
    
    print("global r0 and m")
    r0_m = pd.DataFrame(
        data    = np.empty((n_plates, 2)),
        columns = ("r0", "m"),
        index   = pd.Index(plates, name = "plate")
    )

    ts = np.tile(points, (n_rows-6) * (n_columns-6))
    for p in plates:
        ci = alphas.loc[p, "c i"].values.repeat(n_points)
        r0_m.loc[p] = curve_fit(
            f     = lambda _, r0, m:
                r0 * ci / (ci + np.exp(-m * ts)) * fs[p].reshape(-1),
            xdata = 80085,
            ydata = rho[p, 3:-3, 3:-3].reshape(-1)
        )[0]

    print("local c i")
    c_i = pd.Series(
        data  = np.empty(n_plates * (n_rows-6) * (n_columns-6)),
        index = alphas.index
    )

    for p in plates:
        r0, m = r0_m.loc[p]
        for r, c in product(rows[:-6], columns[:-6]):
            c_i[(p, r, c)] = curve_fit(
                f      = lambda _, ci:
                            r0 * ci / (ci + np.exp(-m * points)) * fs[(p, r, c)],
                xdata  = 80085,
                ydata  = rho[(p, r+3, c+3)],
                bounds = (0, np.inf)
            )[0]

    previous_alphas = alphas.copy()
    alphas.loc[:, "r0 i"] = list(r0_m["r0"].repeat((n_rows-6) * (n_columns-6)))
    alphas.loc[:, "m i"] = list(r0_m["m"].repeat((n_rows-6) * (n_columns-6)))
    alphas.loc[:, "c i"] = list(c_i)

    print("recalculating the alphas and predictions")
    previous_alpha = alpha.copy()
    for p, r, c in product(plates, rows[:-6], columns[:-6]):
        r0, ci, mi = alphas.loc[(p, r, c)]
        alpha[p, r+3, c+3] = r0 * ci / (ci + np.exp(-mi * points))
    
    predictions = np.array([
        dNN(alpha[p], iteration(nt = Nt[p], a = alpha[p], r = rho[p], **params.loc[p]), params.loc[p, "K"], params.loc[p, "kappa"])
        for p in plates
    ])
    
    current_score = lse(predictions.reshape(-1), rho.reshape(-1))
    if current_score < previous_score:
        previous_score = current_score
    else:
        alphas = previous_alphas
        alpha = previous_alpha
        params = previous_params
        break

iteration 1
	plate 1
	plate 2


/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


	plate 3
	plate 4
recalculating the alpha parameters
global r0 and m
local c i
recalculating the alphas and predictions


/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


iteration 2
	plate 1
	plate 2
	plate 3
	plate 4
recalculating the alpha parameters
global r0 and m
local c i
recalculating the alphas and predictions


...

/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


iteration 19
	plate 1
	plate 2
	plate 3
	plate 4
recalculating the alpha parameters
global r0 and m
local c i
recalculating the alphas and predictions


/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


iteration 20
	plate 1
	plate 2
	plate 3
	plate 4
recalculating the alpha parameters
global r0 and m
local c i
recalculating the alphas and predictions


/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


This time, we will store optimal parameters, predictions of $\hat\rho_i(t)$ but also the computed $s(x_i, t)$ :

In [18]:
params.to_csv("diffusion-parameters/optimal.csv", index = False)

In [19]:
predictions = np.array([
    dNN(alpha[p], iteration(nt = Nt[p], a = alpha[p], r = rho[p], **params.loc[p]), params.loc[p, "K"], params.loc[p, "kappa"])
    for p in plates
])

/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


In [20]:
np.save("predictions/level-3_diffusion.npy", predictions)

In [21]:
substrate = np.array([
    iteration(nt = Nt[p], a = alpha[p], r = rho[p], **params.loc[p])
    for p in plates
])

/tmp/ipykernel_492493/4819609.py:2: RuntimeWarning: overflow encountered in exp
  return (1 + np.exp(-K*s)) ** (-kappa)


In [22]:
np.save("substrate/computed.npy", substrate)